In [1]:
import numpy as np
import cardataset as ds
from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
try:
    x = np.load("train_x_aug.npy")
    y = np.load("train_y_aug.npy")
    
    # x = np.load("train_x.npy")
    # y = np.load("train_y.npy")
except: 
    x , y = ds.load_data_augmentation()
    # x , y = ds.load_data()

In [3]:
import random
xy = list(zip(x , y))
random.shuffle(xy)
x , y = zip(*xy)

x = np.array(x)
y = np.array(y)

In [14]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , stratify = y)

In [15]:
y_train = to_categorical(y_train, 12)
y_test = to_categorical(y_test, 12)

In [16]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(6088, 64, 64, 3) (6088, 12)
(1523, 64, 64, 3) (1523, 12)


In [17]:
model = Sequential()
model.add(Conv2D(filters = 32 , kernel_size = (3 , 3) , input_shape = (64 , 64 , 3) , activation = 'relu' , padding = 'same'))
model.add(Conv2D(filters = 32 , kernel_size = (3 , 3) , activation = 'relu' , padding = 'same'))
model.add(MaxPooling2D(pool_size = (2 , 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 64 , kernel_size = (3 , 3) , activation = 'relu' , padding = 'same'))
model.add(Conv2D(filters = 64 , kernel_size = (3 , 3) , activation = 'relu' , padding = 'same'))
model.add(MaxPooling2D(pool_size = (2 , 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(12 , activation = 'softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['acc'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)       

In [ ]:
history = model.fit(x_train , y_train , batch_size = 128 , epochs = 50 , validation_data = (x_test , y_test) , verbose = 1)

Train on 6088 samples, validate on 1523 samples
Epoch 1/50


In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history):
    plt.plot(train_history.history['acc'])
    plt.plot(train_history.history['val_acc'])
    plt.xticks([row for row in range(0, len(train_history.history['acc']))])
    plt.title('Train History')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
show_train_history(history)

In [ ]:
try:
    model.save_weights("car_train.h5")
    print("success")
except:
    print("error")